In [1]:
import xml.etree.ElementTree as ET
from lxml import etree
import json
from tqdm import tqdm

In [2]:
xml_file = './desc2022.xml'
out_file= './desc2022.json'

In [4]:
# get all Tree Number
treeIDset=set()
tree = ET.parse(xml_file)
root = tree.getroot()

TreeNumbers = root.findall(".//TreeNumber")
for treeno in tqdm(TreeNumbers):
    treeIDset.add(treeno.text)

sortedset= sorted(treeIDset)
treeIDlist = list(sortedset)

100%|███████████████████████████████████████████████████████████████████████| 62317/62317 [00:00<00:00, 1639916.44it/s]


In [6]:
# link MeSH Heading with Tree Number
treeDefDic = {}
doc=etree.parse(xml_file)
for elem in tqdm(doc.xpath('//TreeNumber')):
    if elem.text in treeIDlist:
        parent = elem.getparent().getparent().find('.//String')
        treeDefDic[elem.text]=parent.text

100%|███████████████████████████████████████████████████████████████████████████| 62317/62317 [01:27<00:00, 714.55it/s]


In [7]:
def getTreeStructs(elem,treeDefDict):
    treestruts={}
    
    trees = elem.findall('.//TreeNumber')
    for tree in trees:
        # paser tree ids:
        splits = tree.text.split('.')

        treelist = []
        parselen=len(splits)
        for i in reversed(range(parselen)):
            if (i==0): 
                break
            itemdict={}

            id= '.'.join(splits[:i])

            itemdict['id']=id
            name=treeDefDict[id]

            itemdict['name']=name
            treelist.append(itemdict)
        
        treestruts[tree.text]=treelist
    return treestruts

def getSeeAlso(elem):

    terms = elem.findall('.//SeeRelatedDescriptor')
    if terms is None:
        return None

    termList=[]

    for term in terms:
        termDict={}
        term_name = term.findtext(".//DescriptorReferredTo/DescriptorName/String")
        term_id = term.findtext(".//DescriptorReferredTo/DescriptorUI")

        termDict['term_id'] = term_id
        termDict['term_name'] = term_name

        termList.append(termDict)
        
        # no seealso
        if len(termList)==0:
            return None
    return termList

def getSynonyms(elem,name):
    terms = elem.findall('.//Term')
    if terms is None:
        return None

    termList=[]

    for term in terms:
        termDict={}
        term_name = term.findtext(".//String")
        term_id = term.findtext(".//TermUI")

        if (term_name == name):
            continue

        termDict['term_id'] = term_id
        termDict['term_name'] = term_name

        termList.append(termDict)
        if len(termList)==0:
            return None

    return termList 

In [8]:
jsonDict = {}

for elem in tqdm(doc.xpath('//DescriptorRecord')):
    itemDict={}
        
    dui = elem.findtext('.//DescriptorUI')
    name = elem.findtext('.//DescriptorName/String')
    scopeNote = elem.findtext('.//ScopeNote')
    synonyms = getSynonyms(elem,name)
    treeStruts = getTreeStructs(elem,treeDefDic)
    seealso=getSeeAlso(elem)

    itemDict['id']=dui
    itemDict['desc']=scopeNote
    itemDict['treeInfo']=treeStruts
    itemDict['synonyms']=synonyms
    itemDict['seealso']=seealso

    jsonDict[name]=itemDict

with open(out_file, 'w') as outfile:
    json.dump(jsonDict, outfile)

100%|██████████████████████████████████████████████████████████████████████████| 30194/30194 [00:03<00:00, 9796.89it/s]
